In [1]:
import pandas as pd
import os 
import sys
import numpy as np
import sklearn
import time
import copy

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#create single pose dataset for dnn model
class SinglePoseDataset(Dataset):
    
    def __init__(self):
        
        # get data and label from csvget 
        dataset, labels = SinglePoseDataset.ReadPoseData()
        
        self.X = dataset
        self.y = labels
    
    # read dataset from csv file
    @staticmethod
    def ReadPoseData():
        
        # get csv file path
        curr_dir = os.getcwd()
        csv_file_path = os.path.join(curr_dir, 'data/res.csv')
        
        # list for storing data and labels
        data  = []
        label = []
        
        # lenth of sequence
        n_frames = 10
        
        # read csv file
        KP_df = pd.read_csv(csv_file_path)
        
        # convert pos_class to categories
        KP_df['pos_class'] = KP_df['pos_class'].astype('category')
        KP_df['pos_class'] = KP_df['pos_class'].cat.codes

        # skipping (0-3) colomns , return values of all rows and columns from 4 to last
        features = KP_df.iloc[:,4:].values
        #return values of pose_class 
        pose_class = KP_df['pos_class'].values
        # normalize keypoints 
        SinglePoseDataset.normalize_min_(features)
        # append multiple rows to create a sequence of data
        for i in range(features.shape[0]-n_frames):
            data.append(features[i:i+n_frames,...])
            label_sequence = pose_class[i:i+n_frames]
            unique, counts = np.unique(label_sequence, return_counts=True)
            label.append(unique[np.argmax(counts)])
            
        data , label =  np.array(data, dtype = np.float), np.array(label, dtype = np.int_)
        
        return data , label
    
    # min-max normalization to scale the x, y coordinates in range (0-1) 
    @staticmethod
    def normalize_min_(pose:np.ndarray):
        pose = pose.reshape(len(pose),-1,2)
        for i in range(len(pose)):
            xmin = np.min(pose[i,:,0]) 
            ymin = np.min(pose[i,:,1])
            xlen = np.max(pose[i,:,0]) - xmin
            ylen = np.max(pose[i,:,1]) - ymin

            if(xlen==0): pose[i,:,0]=0
            else:
                pose[i,:,0] -= xmin 
                pose[i,:,0] /= xlen

            if(ylen==0): pose[i,:,1]=0
            else:
                pose[i,:,1] -= ymin
                pose[i,:,1] /= ylen
        return pose
    
    # number of rows in the dataset
    def __len__(self):
        
        return len(self.X)
        
    # get a row at an index
    def __getitem__(self, idx):
        
        data  = torch.tensor(self.X[idx], dtype=torch.float) 
        label = torch.tensor(self.y[idx], dtype=torch.long)
        
        return [data,label]
    
    # get indexes for train and test rows
    def get_splits(self, n_test = 0.33):
        
        # determine sizes 
        test_size = round(n_test * len(self.X))
        train_size = len(self.X)-test_size
        
        # calculate the split 
        return random_split(self, [train_size, test_size])
    
    def get_class_labels(self):
        
        labels = ["Fall","Stand", "Tie"]
        
        return labels
    
    def reshape_features(self):
        self.X = self.X.reshape(-1, 340)

In [3]:
class DNN_Single(torch.nn.Module):
    
    def __init__(self, input_dim, class_num, initrange=0.5):
        
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.fc3 = torch.nn.Linear(64,16)
        self.bn3 = torch.nn.BatchNorm1d(16)
        self.fc4 = torch.nn.Linear(16, class_num)
        self.class_num = class_num
        self.init_weights(initrange)
    
    def init_weights(self, initrange):
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
    
    def forward(self, _input):
        
        _fc1 = F.relu(self.fc1(_input))
        
        _bn1 = self.bn1(_fc1)
        
        _fc2 = F.relu(self.fc2(_bn1))
        _bn2 = self.bn2(_fc2)
        
        _fc3 = F.relu(self.fc3(_bn2))
        _bn3 = self.bn3(_fc3)
        
        _fc4 = self.fc4(_bn3)
        
        output = F.softmax(_fc4, dim=1)
        
        return output

In [4]:
def prepare_data():
    
    # load pose dataset
    dataset = SinglePoseDataset()
    
    # reshape from N,10,34 to N,340
    dataset.reshape_features()
    
    # calculate split size
    train,  test = dataset.get_splits()
    
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=16, shuffle=True)
    val_dl  = DataLoader(test, batch_size=1024, shuffle=False)
    
    return {'train':train_dl, 'val':val_dl}, {'train':train, 'val':test}

In [5]:
def save_model(model, optimizer, loss, acc, epoch, save_path):
    
    base_dir = os.path.basename(os.getcwd())
    
    if base_dir =='train':
        parent_dir = os.path.dirname(os.getcwd())
        os.chdir(parent_dir)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
    
    print('SAVING EPOCH %d'%epoch)
    
    SAVE_FILE = os.path.join(save_path,'epoch_%d.pth'%epoch)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'acc':  acc,
            }, SAVE_FILE)
    

In [9]:
def train_model(model, num_epochs=3000):
    
    epoch_losses = []
    epoch_accurs = []
    #get train and val dataloaders
    dataloaders, dataset_sizes = prepare_data()

    # define the optimization
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adadelta(model.parameters())
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.2)
    
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # enumerate over epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-'*10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # St model to training mode
            
            else:
                model.eval() # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0.0
            
            # enumerate over mini_batch
            for i, (inputs, targets)  in enumerate(dataloaders[phase]):
                inputs  = inputs.to(device)
                targets = targets.to(device)
                
                # clear the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train pahse
                with torch.set_grad_enabled(phase=='train'):
                    
                    # compute model outputs
                    outputs = model(inputs)
                    
                    # calculate outputs
                    _, preds = torch.max(outputs, dim=1)
                    
                    # calculate the loss
                    loss = criterion(outputs, targets)
                    
                    # backward + optimize only ig in training phase
                    if phase == 'train':
                        # calculate gradient
                        loss.backward()
                    
                        # update model weights
                        optimizer.step()
                
                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==targets)
            
            if phase=='train':
                scheduler.step()
            
            epoch_loss = running_loss / dataset_sizes[phase].__len__()
            epoch_acc  = running_corrects.double() / dataset_sizes[phase].__len__()
            
            epoch_losses.append(epoch_loss)
            epoch_accurs.append(epoch_acc)
            
            print('{} Loss: {:.4f} Acc {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase== 'val' and epoch_acc> best_acc and epoch%200==0 and epoch!=0:
                best_acc = epoch_acc
                save_model(model, optimizer, epoch_loss, epoch_acc, epoch, save_path=r'model/act_dnnSingle')
        print()
    
    time_elapsed = time.time() - since
    
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed //60, time_elapsed %60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    return {'losses':epoch_losses, 'acc':epoch_accurs}

In [7]:
DNN_model = DNN_Single(input_dim=340,class_num=3).to(device)

In [8]:
history = train_model(DNN_model, num_epochs=1000)

Epoch 0/999
----------
train Loss: 0.6570 Acc 0.8996
val Loss: 0.6078 Acc 0.9438

Epoch 1/999
----------
train Loss: 0.6151 Acc 0.9358
val Loss: 0.5960 Acc 0.9554

Epoch 2/999
----------
train Loss: 0.6043 Acc 0.9466
val Loss: 0.5938 Acc 0.9577

Epoch 3/999
----------
train Loss: 0.5984 Acc 0.9524
val Loss: 0.5904 Acc 0.9610

Epoch 4/999
----------
train Loss: 0.5924 Acc 0.9585
val Loss: 0.5871 Acc 0.9643

Epoch 5/999
----------
train Loss: 0.5896 Acc 0.9614
val Loss: 0.5823 Acc 0.9691

Epoch 6/999
----------
train Loss: 0.5873 Acc 0.9638
val Loss: 0.5805 Acc 0.9705

Epoch 7/999
----------
train Loss: 0.5852 Acc 0.9660
val Loss: 0.5808 Acc 0.9711

Epoch 8/999
----------
train Loss: 0.5831 Acc 0.9677
val Loss: 0.5774 Acc 0.9739

Epoch 9/999
----------
train Loss: 0.5822 Acc 0.9695
val Loss: 0.5799 Acc 0.9711

Epoch 10/999
----------
train Loss: 0.5784 Acc 0.9726
val Loss: 0.5743 Acc 0.9767

Epoch 11/999
----------
train Loss: 0.5750 Acc 0.9761
val Loss: 0.5729 Acc 0.9782

Epoch 12/999
-

train Loss: 0.5568 Acc 0.9945
val Loss: 0.5654 Acc 0.9857

Epoch 100/999
----------
train Loss: 0.5560 Acc 0.9953
val Loss: 0.5641 Acc 0.9868

Epoch 101/999
----------
train Loss: 0.5555 Acc 0.9960
val Loss: 0.5648 Acc 0.9868

Epoch 102/999
----------
train Loss: 0.5551 Acc 0.9962
val Loss: 0.5646 Acc 0.9871

Epoch 103/999
----------
train Loss: 0.5550 Acc 0.9964
val Loss: 0.5650 Acc 0.9862

Epoch 104/999
----------
train Loss: 0.5550 Acc 0.9963
val Loss: 0.5646 Acc 0.9869

Epoch 105/999
----------
train Loss: 0.5555 Acc 0.9959
val Loss: 0.5647 Acc 0.9866

Epoch 106/999
----------
train Loss: 0.5556 Acc 0.9959
val Loss: 0.5635 Acc 0.9877

Epoch 107/999
----------
train Loss: 0.5550 Acc 0.9965
val Loss: 0.5641 Acc 0.9868

Epoch 108/999
----------
train Loss: 0.5554 Acc 0.9960
val Loss: 0.5650 Acc 0.9863

Epoch 109/999
----------
train Loss: 0.5551 Acc 0.9962
val Loss: 0.5642 Acc 0.9871

Epoch 110/999
----------
train Loss: 0.5546 Acc 0.9968
val Loss: 0.5654 Acc 0.9857

Epoch 111/999
---

train Loss: 0.5538 Acc 0.9975
val Loss: 0.5637 Acc 0.9875

Epoch 198/999
----------
train Loss: 0.5533 Acc 0.9981
val Loss: 0.5639 Acc 0.9872

Epoch 199/999
----------
train Loss: 0.5537 Acc 0.9976
val Loss: 0.5636 Acc 0.9879

Epoch 200/999
----------
train Loss: 0.5539 Acc 0.9976
val Loss: 0.5625 Acc 0.9888

Epoch 201/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5642 Acc 0.9871

Epoch 202/999
----------
train Loss: 0.5542 Acc 0.9971
val Loss: 0.5632 Acc 0.9875

Epoch 203/999
----------
train Loss: 0.5539 Acc 0.9974
val Loss: 0.5632 Acc 0.9881

Epoch 204/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5625 Acc 0.9884

Epoch 205/999
----------
train Loss: 0.5541 Acc 0.9973
val Loss: 0.5640 Acc 0.9866

Epoch 206/999
----------
train Loss: 0.5540 Acc 0.9973
val Loss: 0.5634 Acc 0.9880

Epoch 207/999
----------
train Loss: 0.5539 Acc 0.9974
val Loss: 0.5622 Acc 0.9891

Epoch 208/999
----------
train Loss: 0.5542 Acc 0.9973
val Loss: 0.5628 Acc 0.9882

Epoch 209/999
---

train Loss: 0.5539 Acc 0.9976
val Loss: 0.5634 Acc 0.9881

Epoch 296/999
----------
train Loss: 0.5537 Acc 0.9977
val Loss: 0.5617 Acc 0.9893

Epoch 297/999
----------
train Loss: 0.5540 Acc 0.9975
val Loss: 0.5632 Acc 0.9881

Epoch 298/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5633 Acc 0.9880

Epoch 299/999
----------
train Loss: 0.5533 Acc 0.9982
val Loss: 0.5640 Acc 0.9870

Epoch 300/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5628 Acc 0.9886

Epoch 301/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5636 Acc 0.9875

Epoch 302/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5622 Acc 0.9889

Epoch 303/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5629 Acc 0.9884

Epoch 304/999
----------
train Loss: 0.5532 Acc 0.9983
val Loss: 0.5619 Acc 0.9894

Epoch 305/999
----------
train Loss: 0.5535 Acc 0.9978
val Loss: 0.5650 Acc 0.9861

Epoch 306/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5639 Acc 0.9872

Epoch 307/999
---

train Loss: 0.5532 Acc 0.9982
val Loss: 0.5638 Acc 0.9874

Epoch 394/999
----------
train Loss: 0.5538 Acc 0.9977
val Loss: 0.5643 Acc 0.9871

Epoch 395/999
----------
train Loss: 0.5533 Acc 0.9980
val Loss: 0.5634 Acc 0.9875

Epoch 396/999
----------
train Loss: 0.5534 Acc 0.9980
val Loss: 0.5634 Acc 0.9879

Epoch 397/999
----------
train Loss: 0.5535 Acc 0.9978
val Loss: 0.5631 Acc 0.9879

Epoch 398/999
----------
train Loss: 0.5533 Acc 0.9982
val Loss: 0.5633 Acc 0.9880

Epoch 399/999
----------
train Loss: 0.5532 Acc 0.9983
val Loss: 0.5635 Acc 0.9876

Epoch 400/999
----------
train Loss: 0.5536 Acc 0.9977
val Loss: 0.5635 Acc 0.9878

Epoch 401/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5632 Acc 0.9880

Epoch 402/999
----------
train Loss: 0.5536 Acc 0.9979
val Loss: 0.5646 Acc 0.9870

Epoch 403/999
----------
train Loss: 0.5537 Acc 0.9977
val Loss: 0.5639 Acc 0.9872

Epoch 404/999
----------
train Loss: 0.5534 Acc 0.9981
val Loss: 0.5629 Acc 0.9885

Epoch 405/999
---

...

train Loss: 0.5538 Acc 0.9976
val Loss: 0.5643 Acc 0.9869

Epoch 590/999
----------
train Loss: 0.5538 Acc 0.9976
val Loss: 0.5642 Acc 0.9875

Epoch 591/999
----------
train Loss: 0.5533 Acc 0.9981
val Loss: 0.5635 Acc 0.9880

Epoch 592/999
----------
train Loss: 0.5535 Acc 0.9980
val Loss: 0.5635 Acc 0.9875

Epoch 593/999
----------
train Loss: 0.5533 Acc 0.9981
val Loss: 0.5630 Acc 0.9883

Epoch 594/999
----------
train Loss: 0.5530 Acc 0.9985
val Loss: 0.5631 Acc 0.9882

Epoch 595/999
----------
train Loss: 0.5531 Acc 0.9983
val Loss: 0.5627 Acc 0.9882

Epoch 596/999
----------
train Loss: 0.5537 Acc 0.9978
val Loss: 0.5629 Acc 0.9884

Epoch 597/999
----------
train Loss: 0.5537 Acc 0.9977
val Loss: 0.5634 Acc 0.9880

Epoch 598/999
----------
train Loss: 0.5536 Acc 0.9979
val Loss: 0.5642 Acc 0.9869

Epoch 599/999
----------
train Loss: 0.5529 Acc 0.9987
val Loss: 0.5629 Acc 0.9883

Epoch 600/999
----------
train Loss: 0.5532 Acc 0.9981
val Loss: 0.5631 Acc 0.9881

Epoch 601/999
---

train Loss: 0.5530 Acc 0.9985
val Loss: 0.5628 Acc 0.9884

Epoch 688/999
----------
train Loss: 0.5539 Acc 0.9976
val Loss: 0.5632 Acc 0.9880

Epoch 689/999
----------
train Loss: 0.5533 Acc 0.9981
val Loss: 0.5638 Acc 0.9875

Epoch 690/999
----------
train Loss: 0.5536 Acc 0.9977
val Loss: 0.5632 Acc 0.9880

Epoch 691/999
----------
train Loss: 0.5535 Acc 0.9980
val Loss: 0.5633 Acc 0.9876

Epoch 692/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5633 Acc 0.9881

Epoch 693/999
----------
train Loss: 0.5532 Acc 0.9983
val Loss: 0.5628 Acc 0.9884

Epoch 694/999
----------
train Loss: 0.5534 Acc 0.9980
val Loss: 0.5631 Acc 0.9881

Epoch 695/999
----------
train Loss: 0.5534 Acc 0.9979
val Loss: 0.5630 Acc 0.9882

Epoch 696/999
----------
train Loss: 0.5530 Acc 0.9985
val Loss: 0.5631 Acc 0.9883

Epoch 697/999
----------
train Loss: 0.5538 Acc 0.9976
val Loss: 0.5636 Acc 0.9875

Epoch 698/999
----------
train Loss: 0.5538 Acc 0.9975
val Loss: 0.5622 Acc 0.9895

Epoch 699/999
---

train Loss: 0.5533 Acc 0.9982
val Loss: 0.5620 Acc 0.9892

Epoch 786/999
----------
train Loss: 0.5531 Acc 0.9983
val Loss: 0.5632 Acc 0.9881

Epoch 787/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5636 Acc 0.9875

Epoch 788/999
----------
train Loss: 0.5534 Acc 0.9981
val Loss: 0.5637 Acc 0.9875

Epoch 789/999
----------
train Loss: 0.5538 Acc 0.9977
val Loss: 0.5641 Acc 0.9868

Epoch 790/999
----------
train Loss: 0.5536 Acc 0.9979
val Loss: 0.5634 Acc 0.9878

Epoch 791/999
----------
train Loss: 0.5534 Acc 0.9980
val Loss: 0.5635 Acc 0.9878

Epoch 792/999
----------
train Loss: 0.5531 Acc 0.9984
val Loss: 0.5633 Acc 0.9881

Epoch 793/999
----------
train Loss: 0.5535 Acc 0.9980
val Loss: 0.5640 Acc 0.9873

Epoch 794/999
----------
train Loss: 0.5532 Acc 0.9982
val Loss: 0.5642 Acc 0.9869

Epoch 795/999
----------
train Loss: 0.5537 Acc 0.9977
val Loss: 0.5650 Acc 0.9859

Epoch 796/999
----------
train Loss: 0.5536 Acc 0.9977
val Loss: 0.5632 Acc 0.9879

Epoch 797/999
---

train Loss: 0.5533 Acc 0.9981
val Loss: 0.5636 Acc 0.9874

Epoch 884/999
----------
train Loss: 0.5535 Acc 0.9980
val Loss: 0.5643 Acc 0.9868

Epoch 885/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5653 Acc 0.9860

Epoch 886/999
----------
train Loss: 0.5537 Acc 0.9977
val Loss: 0.5639 Acc 0.9873

Epoch 887/999
----------
train Loss: 0.5536 Acc 0.9978
val Loss: 0.5625 Acc 0.9886

Epoch 888/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5638 Acc 0.9875

Epoch 889/999
----------
train Loss: 0.5534 Acc 0.9979
val Loss: 0.5626 Acc 0.9886

Epoch 890/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5645 Acc 0.9868

Epoch 891/999
----------
train Loss: 0.5530 Acc 0.9985
val Loss: 0.5630 Acc 0.9882

Epoch 892/999
----------
train Loss: 0.5530 Acc 0.9985
val Loss: 0.5642 Acc 0.9875

Epoch 893/999
----------
train Loss: 0.5536 Acc 0.9979
val Loss: 0.5636 Acc 0.9876

Epoch 894/999
----------
train Loss: 0.5533 Acc 0.9981
val Loss: 0.5633 Acc 0.9879

Epoch 895/999
---

train Loss: 0.5536 Acc 0.9977
val Loss: 0.5640 Acc 0.9872

Epoch 982/999
----------
train Loss: 0.5537 Acc 0.9976
val Loss: 0.5637 Acc 0.9876

Epoch 983/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5630 Acc 0.9880

Epoch 984/999
----------
train Loss: 0.5530 Acc 0.9985
val Loss: 0.5631 Acc 0.9882

Epoch 985/999
----------
train Loss: 0.5534 Acc 0.9981
val Loss: 0.5635 Acc 0.9879

Epoch 986/999
----------
train Loss: 0.5538 Acc 0.9975
val Loss: 0.5633 Acc 0.9879

Epoch 987/999
----------
train Loss: 0.5528 Acc 0.9986
val Loss: 0.5639 Acc 0.9872

Epoch 988/999
----------
train Loss: 0.5534 Acc 0.9979
val Loss: 0.5633 Acc 0.9878

Epoch 989/999
----------
train Loss: 0.5532 Acc 0.9983
val Loss: 0.5638 Acc 0.9876

Epoch 990/999
----------
train Loss: 0.5534 Acc 0.9979
val Loss: 0.5645 Acc 0.9871

Epoch 991/999
----------
train Loss: 0.5535 Acc 0.9979
val Loss: 0.5640 Acc 0.9871

Epoch 992/999
----------
train Loss: 0.5532 Acc 0.9984
val Loss: 0.5635 Acc 0.9876

Epoch 993/999
---